In [51]:
import pandas as pd 
import numpy as np
import pickle
import gc
from datetime import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from xgboost import XGBClassifier
from bayes_opt import BayesianOptimization

## on mac run `brew install lightgbm` 
## anythign else refer here https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html#apple-clang
from lightgbm import LGBMClassifier

In [52]:
train_df = pd.read_csv("./data/application_train.csv")
test_df = pd.read_csv("./data/application_test.csv")


In [53]:
## clean up some rows
train_df['DAYS_EMPLOYED'][train_df['DAYS_EMPLOYED'] == 365243] = np.nan
test_df['DAYS_EMPLOYED'][test_df['DAYS_EMPLOYED'] == 365243] = np.nan
train_df['OBS_30_CNT_SOCIAL_CIRCLE'][train_df['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
train_df['OBS_60_CNT_SOCIAL_CIRCLE'][train_df['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
test_df['OBS_30_CNT_SOCIAL_CIRCLE'][test_df['OBS_30_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
test_df['OBS_60_CNT_SOCIAL_CIRCLE'][test_df['OBS_60_CNT_SOCIAL_CIRCLE'] > 30] == np.nan
train_df = train_df[train_df['CODE_GENDER'] != 'XNA']

## remove rows with only 1 distinct value
empty_columns = []
for col in train_df.columns:
    if len(train_df[col].unique()) <=1:
        empty_columns.append(col)
train_df = train_df.drop(empty_columns, axis = 1)
test_df = test_df.drop(empty_columns, axis = 1)

<ipython-input-53-286c6ff8d1a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['DAYS_EMPLOYED'][train_df['DAYS_EMPLOYED'] == 365243] = np.nan
<ipython-input-53-286c6ff8d1a2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['DAYS_EMPLOYED'][test_df['DAYS_EMPLOYED'] == 365243] = np.nan


In [54]:
## other pre-processing
train_df["DAYS_BIRTH"] = train_df["DAYS_BIRTH"] * -1 / 365
test_df["DAYS_BIRTH"] = test_df["DAYS_BIRTH"] * -1 / 365
categorical_columns = train_df.dtypes[train_df.dtypes == "object"].index.tolist()
train_df[categorical_columns] = train_df[categorical_columns].fillna("XNA")
test_df[categorical_columns] = test_df[categorical_columns].fillna("XNA")
train_df["REGION_RATING_CLIENT"] = train_df["REGION_RATING_CLIENT"].astype("object")
train_df["REGION_RATING_CLIENT_W_CITY"] = train_df[
    "REGION_RATING_CLIENT_W_CITY"
].astype("object")
test_df["REGION_RATING_CLIENT"] = test_df["REGION_RATING_CLIENT"].astype("object")
test_df["REGION_RATING_CLIENT_W_CITY"] = test_df["REGION_RATING_CLIENT_W_CITY"].astype(
    "object"
)


In [55]:
bad_cols = "NAME_CONTRACT_TYPE, CODE_GENDER, FLAG_OWN_CAR, FLAG_OWN_REALTY, NAME_TYPE_SUITE, NAME_INCOME_TYPE, NAME_EDUCATION_TYPE, NAME_FAMILY_STATUS, NAME_HOUSING_TYPE, OCCUPATION_TYPE, REGION_RATING_CLIENT, REGION_RATING_CLIENT_W_CITY, WEEKDAY_APPR_PROCESS_START, ORGANIZATION_TYPE, FONDKAPREMONT_MODE, HOUSETYPE_MODE, WALLSMATERIAL_MODE, EMERGENCYSTATE_MODE".split(", ")
for c in bad_cols:
    train_df[c] = train_df[c].astype("category")
    test_df[c] = test_df[c].astype("category")


In [56]:
print(train_df.shape)
print(test_df.shape)
target_df = train_df.pop("TARGET")

(307507, 122)
(48744, 121)


In [57]:
## using LightGBM for recursive feature selection
def cut_down_features(train_df, test_df):
    num_folds = 3
    impt_cols = set()
    score = 1
    i = 1
    while score > 0.72:
        selection_data = train_df.drop(list(impt_cols), axis=1)
        fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=33)
        score = 0
        model_feature_importance = np.zeros_like(selection_data.columns)
        for fold_num, (train_indices, val_indices) in enumerate(
            fold.split(selection_data, target_df), 1
        ):
            x_train = selection_data.iloc[train_indices]
            x_val = selection_data.iloc[val_indices]
            y_train = target_df.iloc[train_indices]
            y_val = target_df.iloc[val_indices]
            lg = LGBMClassifier(n_jobs=-1, random_state=69)
            lg.fit(x_train, y_train)
            model_feature_importance += lg.feature_importances_ / num_folds
            score += roc_auc_score(y_val, lg.predict_proba(x_val)[:, 1]) / num_folds
        imp_cols_indices = np.where(np.abs(model_feature_importance) > 0)
        cols_imp = train_df.columns[imp_cols_indices]
        if score > 0.7:
            impt_cols.update(cols_imp)
        i += 1
    impt_cols = list(impt_cols)
    train_df = train_df[impt_cols]
    test_df = test_df[impt_cols]
    with open("final_cols.pkl", "wb") as f:
        pickle.dump(train_df.columns.tolist(), f)
    gc.collect()
    return train_df, test_df


reduced_train_df, reduced_test_df = cut_down_features(train_df, test_df)


In [58]:
print(len(train_df.columns))
print(len(reduced_train_df.columns))
#lmao

121
108


In [74]:
model = XGBClassifier(enable_categorical=True)

In [67]:
def xgb_evaluation(
    max_depth,
    min_child_weight,
    gamma,
    subsample,
    colsample_bytree,
    colsample_bylevel,
    colsample_bynode,
    reg_alpha,
    reg_lambda,
):

    params = {
        "learning_rate": 0.01,
        "n_estimators": 10000,
        "tree_method": "gpu_hist",
        "gpu_id": 0,
        "max_depth": int(round(max_depth)),
        "min_child_weight": int(round(min_child_weight)),
        "subsample": subsample,
        "gamma": gamma,
        "colsample_bytree": colsample_bytree,
        "colsample_bylevel": colsample_bylevel,
        "colsample_bynode": colsample_bynode,
        "reg_alpha": reg_alpha,
        "reg_lambda": reg_lambda,
        "random_state": 51412,
        "enable_categorical": True
    }

    # defining the Cross-Validation Strategry
    stratified_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=33)
    cv_preds = np.zeros(reduced_train_df.shape[0])

    # iterating over each fold, training the model, and making Out of Fold Predictions
    for train_indices, cv_indices in stratified_cv.split(reduced_train_df, target_df):

        x_tr = reduced_train_df.iloc[train_indices]
        y_tr = target_df.iloc[train_indices]
        x_cv = reduced_train_df.iloc[cv_indices]
        y_cv = target_df.iloc[cv_indices]

        xgbc = XGBClassifier(**params)
        xgbc.fit(
            x_tr,
            y_tr,
            eval_set=[(x_cv, y_cv)],
            eval_metric="auc",
            verbose=False,
            early_stopping_rounds=200,
        )

        cv_preds[cv_indices] = xgbc.predict_proba(
            x_cv, ntree_limit=xgbc.get_booster().best_ntree_limit
        )[:, 1]
        gc.collect()

    return roc_auc_score(target_df, cv_preds)

bopt_xgb = BayesianOptimization(
    xgb_evaluation,
    {
        "max_depth": (5, 15),
        "min_child_weight": (5, 80),
        "gamma": (0.2, 1),
        "subsample": (0.5, 1),
        "colsample_bytree": (0.5, 1),
        "colsample_bylevel": (0.3, 1),
        "colsample_bynode": (0.3, 1),
        "reg_alpha": (0.001, 0.3),
        "reg_lambda": (0.001, 0.3),
    },
    random_state=55,
)
bopt_xgb.maximize(n_iter=6, init_points=4)


|   iter    |  target   | colsam... | colsam... | colsam... |   gamma   | max_depth | min_ch... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------


/usr/local/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.NAME_HOUSING_TYPE, FLAG_OWN_REALTY, REGION_RATING_CLIENT_W_CITY, REGION_RATING_CLIENT, ORGANIZATION_TYPE, FONDKAPREMONT_MODE, OCCUPATION_TYPE, NAME_EDUCATION_TYPE, CODE_GENDER, NAME_FAMILY_STATUS, NAME_TYPE_SUITE, NAME_CONTRACT_TYPE, WALLSMATERIAL_MODE, NAME_INCOME_TYPE, HOUSETYPE_MODE, FLAG_OWN_CAR, WEEKDAY_APPR_PROCESS_START